In [1]:
#  TriQuintic Interpolator

# Section 0

import numpy as np
import numpy.linalg as la

# Sympy is used for the symbolic expressions.
from sympy import *


In [2]:

#  Section 1: Creation of Polynomial Components

# Declaring Symbols for Sympy
x, y, z = symbols('x y z')    

#  List of Variable Interactions for creating the polynomial
variables = []  

# Empty list for storing index
index = []

degree = 5

#  Loop for Generating Variable Interaction
for k in range(0, degree + 1):                 # +1 because Python lists are shifted back so this corrects for it.
    for j in range(0, degree + 1):
        for i in range(0, degree + 1):
            variables.append( x**i * y**j * z**k )  #Variable factors
                    
            index.append(str(i) + str(j) + str(k))  #Index for Coeffs

# Coefficients
coeffs = ['a' + x for x in index]


#  Section 2:  Construction of Polynomials


def create_polynomial(coeffiecients:list , variable_interactions:list):
    """
    Takes in a list of coeffecients and a list of variable interaction
    and assembles a polynomial that is the dot produce of the two.
    
    Converts the polynomial to Numpy object.
    """
    # Create the polynomial using the dot product
    polynomial = Matrix([variable_interactions])* Transpose(Matrix([coeffiecients]))
    
    # Convert to Numpy
    return lambdify((x,y,z), expand(polynomial[0]),  modules='numpy')



def create_derivative(polynomial, variables:tuple):
    """
    Takes in the order 0 polynomial, and a tuple of variables in order of differentiation.
    
    Returns the derivative of the polynomial, that can be evaluated numerically.
    """
    # Differentiate and convert to evaluatable function.
    derivative = lambdify((x,y,z), (diff(polynomial, *variables)),  modules='numpy')

    return derivative
    


# Construction of polynomials

#  Construction of Polynomial p(x,y)
p = create_polynomial(coeffs, variables)


###########################################
# Order 1
# x derivative
px = create_derivative(  p(x,y,z),   (x,)  )

# y derivative
py = create_derivative(p(x,y,z), (y,))

# z derivative
pz = create_derivative(p(x,y,z), (z,))


###########################################
# Order 2
# The xx derv
pxx = create_derivative(p(x,y,z), (x,x) )

# The xy derv
pxy = create_derivative(p(x,y,z), (x,y))

# The xz derv
pxz = create_derivative(p(x,y,z), (x,z))

# The yz derv
pyz = create_derivative(p(x,y,z), (y,z))

# The yy derv
pyy = create_derivative(p(x,y,z), (y,y))

# The zz derv
pzz = create_derivative(p(x,y,z), (z,z))


###########################################
# Order 3
# xxx derv
pxxx = create_derivative(p(x,y,z), (x,x,x))

# xxy derv
pxxy = create_derivative(p(x,y,z), (x,x,y))

# xxz derv
pxxz = create_derivative(p(x,y,z), (x,x,z))

# xyy derv
pxyy = create_derivative(p(x,y,z), (x,y,y))

# pxyz(x,y,z)  The xyz derv
pxyz = create_derivative(p(x,y,z), (x,y,z))

# xzz derv
pxzz = create_derivative(p(x,y,z), (x,z,z))

# yyy derv
pyyy = create_derivative(p(x,y,z), (y,y,y))

# yyz derv
pyyz = create_derivative(p(x,y,z), (y,y,z))

# yzz derv
pyzz = create_derivative(p(x,y,z), (y,z,z))

# zzz derv
pzzz = create_derivative(p(x,y,z), (z,z,z))


###########################################
# Order 4
# xxxx derv
pxxxx = create_derivative(p(x,y,z), (x,x,x,x))

# yyyy derv
pyyyy = create_derivative(p(x,y,z), (y,y,y,y))

# zzzz derv
pzzzz = create_derivative(p(x,y,z), (z,z,z,z))


# xxxy derv
pxxxy = create_derivative(p(x,y,z), (x,x,x,y))

# xxxz derv
pxxxz = create_derivative(p(x,y,z), (x,x,x,z))

# xyyy derv
pxyyy = create_derivative(p(x,y,z), (x,y,y,y))

# yyyz derv
pyyyz = create_derivative(p(x,y,z), (y,y,y,z))

# xzzz derv
pxzzz = create_derivative(p(x,y,z), (x,z,z,z))

# xxyy derv
pxxyy = create_derivative(p(x,y,z), (x,x,y,y))

# xxzz derv
pxxzz = create_derivative(p(x,y,z), (x,x,z,z))

# yyzz derv
pyyzz = create_derivative(p(x,y,z), (y,y,z,z))


# xyzz derv
pxyzz = create_derivative(p(x,y,z), (x,y,z,z))

# xyyz derv
pxyyz = create_derivative(p(x,y,z), (x,y,y,z))

# xxyz derv
pxxyz = create_derivative(p(x,y,z), (x,x,y,z))


###########################################
# Order 5
# xxyyz
pxxyyz = create_derivative(p(x,y,z), (x,x,y,y,z))

#xxyzz
pxxyzz = create_derivative(p(x,y,z), (x,x,y,z,z))

# xyyzz
pxyyzz = create_derivative(p(x,y,z), (x,y,y,z,z))


###########################################
# Order 6
#xxyyzz

pxxyyzz = create_derivative(p(x,y,z), (x,x,y,y,z,z))

#  Section 3:  Matrix Creation Functions  

# The Row creating function.  

#  Takes a function evaluated at a point, extracts coefficients
#  Combines everything from above into one process.

def rowcreate(polynomial)->list:
    """
    Takes in a polynomial evaluated at a point.
    
    Separates the numerical coefficients from the alpha coefficients.
    
    Returns a list of numerical coefficients which will later be a single row of the coefficient matrix.
    """
    # Convert the polynomial to string.
    polynomial = str(polynomial)
    
    # Separate the terms of the polynomial
    terms = polynomial.split('+')
    
    # Dictionary for storing alphas and the numeric coefficient.
    alphco = {}    
    for i in range(0,len(terms)):
        if '*' in terms[i]:     # Set dictionary entry to "a coeff" = value
            alphco[terms[i].split('*')[1].strip()] = float(terms[i].split('*')[0])
        else:
            alphco[terms[i].strip()] = 1
    
    # Row of zeros as template.
    row = [0] * len(variables)                  
    for key in alphco:   # Replace zeros with entries 
        row[coeffs.index(key)] = alphco[key]
        
    return row


#  Matrix Creation Function

def create_unfitted_matrix(function_names:list, points:list)->np.ndarray:
    """
    Takes in a list of a polynomial and its derivatives, as well as a list
    of the eight corners as tupules.
    
    Evaluates all the functions at all the corners, each of which gives us a 
    row of coefficients.
    
    Returns a Numpy matrix of coefficients which will later be solved.
    """
    #  The Matrix
    rows_list = []
    
    # We evaluate the polynomial and its derivatives across the eight corners.
    # We then create the row of coefficients for the matrix.
    for f in function_names: 
        for point in points:
            rows_list.append(rowcreate(f(*point)))
           
    # Take the list of rows of coefficients and create the matrix.
    return np.array(rows_list)



# Lists of possible functions

# Order followed by a number indicates all function present.

order_zero_one_two = [p, px, py, pz, pxx, pxy, pxz, pyy, pyz, pzz]  # Length 10


order_three_ex = [pxxy, pxxz, pxyy, pxyz, pyyz, pxzz, pyzz]     # Length 7,     Ex triplets of one variable.

order_four_ex = [pxxyy, pxxzz, pyyzz, pxxyz, pxyyz, pxyzz]      # Length 6

order_five_ex = [pxxyyz, pxxyzz, pxyyzz]                       # Length 3

order_six_ex = [pxxyyzz]                                      # Length 1



# Mesh Size Parameters:  Choosing where we pin down the function.

# The eight outer corners of the unit cube.
unit_cube = [(k, j, i)  for i in [0,1] for j in [0,1] for k in [0,1]]



In [ ]:
# Inner Box
# Option 1: Quarter box
quarter_box = [(k, j, i)  for i in [1/4, 3/4] for j in [1/4, 3/4] for k in [1/4, 3/4]]

# Option 2: Thirds box
thirds_box = [(k, j, i)  for i in [1/3, 2/3] for j in [1/3, 2/3] for k in [1/3, 2/3]]


In [ ]:
unit_cube

In [ ]:
quarter_box

In [ ]:
thirds_box

In [3]:
#  Matrix Inspection Function

def matrix_inspect(matrix)->bool:
    """
    Prints info about the matrix.
    
    Returns True if the system has one unique solution.
    """
    print(matrix)

    print('\n The dimensions of the matrix are: ',  np.shape(matrix))

    print('\n Rank: ', la.matrix_rank(matrix))
    
    if (np.shape(matrix)[0]==la.matrix_rank(matrix)) & (np.shape(matrix)[1]==la.matrix_rank(matrix)):
        return True
    
    else: 
        return False

In [ ]:
# Case 1:  Uses outer and inner box.  

# Choosing the functions to use

outer_functions =  order_zero_one_two + order_three_ex  # 17 eqns

inner_functions = order_zero_one_two   # 10 eqns


# Matrix Creation Process
top_matrix = create_unfitted_matrix(outer_functions, unit_cube)

# Bottom matrix.
bottom_matrix = create_unfitted_matrix(inner_functions, quarter_box)


# Create the matrix by concatenating the top and bottom.
case1_matrix = np.concatenate((top_matrix, bottom_matrix), axis=0)

# Inspect the matrix.
(matrix_inspect(case1_matrix))


In [ ]:
np.amax(case1_matrix)

In [ ]:
la.cond(case1_matrix)

In [4]:
# Case 2

# Uses only the outer box.

case2_polynomials = order_zero_one_two + order_three_ex + order_four_ex + order_five_ex + order_six_ex 

case2_matrix = create_unfitted_matrix(case2_polynomials, unit_cube)

matrix_inspect(case2_matrix)


[[1.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+00 1.0e+00 1.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [1.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 ...
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00 ... 2.4e+03 4.8e+03 8.0e+03]]

 The dimensions of the matrix are:  (216, 216)

 Rank:  216


True

In [ ]:
np.amax(case2_matrix)

In [ ]:
la.cond(case2_matrix)

In [5]:
####################################################################

# Testing Section

#  Test Function number 1

def f(x,y,z):
    return exp(-(x**2 + y**2 + z**2))    # e^-(x^2 + y^2 + z^2)


#  Other test function
def h(x,y,z):
    1/(x**2 + y**2 + z**2)**.5
    
    
###########################################
# Order 1
# x derivative
fx = create_derivative(f(x,y,z), (x,))

# y derivative
fy = create_derivative(f(x,y,z), (y,))

# z derivative
fz = create_derivative(f(x,y,z), (z,))


###########################################
# Order 2
# The xx derv
fxx = create_derivative(f(x,y,z), (x,x))

# The xy derv
fxy = create_derivative(f(x,y,z), (x,y))

# The xz derv
fxz = create_derivative(f(x,y,z), (x,z))

# The yz derv
fyz = create_derivative(f(x,y,z), (y,z))

# The yy derv
fyy = create_derivative(f(x,y,z), (y,y))

# The zz derv
fzz = create_derivative(f(x,y,z), (z,z))


###########################################
# Order 3
# xxx derv
fxxx = create_derivative(f(x,y,z), (x,x,x))

# xxy derv
fxxy = create_derivative(f(x,y,z), (x,x,y))

# xxz derv
fxxz = create_derivative(f(x,y,z), (x,x,z))

# xyy derv
fxyy = create_derivative(f(x,y,z), (x,y,y))

# pxyz(x,y,z)  The xyz derv
fxyz = create_derivative(f(x,y,z), (x,y,z))

# xzz derv
fxzz = create_derivative(f(x,y,z), (x,z,z))

# yyy derv
fyyy = create_derivative(f(x,y,z), (y,y,y))

# yyz derv
fyyz = create_derivative(f(x,y,z), (y,y,z))

# yzz derv
fyzz = create_derivative(f(x,y,z), (y,z,z))

# zzz derv
fzzz = create_derivative(f(x,y,z), (z,z,z))


###########################################
# Order 4
# xxxx derv
fxxxx = create_derivative(f(x,y,z), (x,x,x,x))

# yyyy derv
fyyyy = create_derivative(f(x,y,z), (y,y,y,y))

# zzzz derv
fzzzz = create_derivative(f(x,y,z), (z,z,z,z))


# xxxy derv
fxxxy = create_derivative(f(x,y,z), (x,x,x,y))

# xxxz derv
fxxxz = create_derivative(f(x,y,z), (x,x,x,z))

# xyyy derv
fxyyy = create_derivative(f(x,y,z), (x,y,y,y))

# yyyz derv
fyyyz = create_derivative(f(x,y,z), (y,y,y,z))

# xzzz derv
fxzzz = create_derivative(f(x,y,z), (x,z,z,z))

# xxyy derv
fxxyy = create_derivative(f(x,y,z), (x,x,y,y))

# xxzz derv
fxxzz = create_derivative(f(x,y,z), (x,x,z,z))

# yyzz derv
fyyzz = create_derivative(f(x,y,z), (y,y,z,z))


# xyzz derv
fxyzz = create_derivative(f(x,y,z), (x,y,z,z))

# xyyz derv
fxyyz = create_derivative(f(x,y,z), (x,y,y,z))

# xxyz derv
fxxyz = create_derivative(f(x,y,z), (x,x,y,z))


###########################################
# Order 5
# xxyyz
fxxyyz = create_derivative(f(x,y,z), (x,x,y,y,z))

#xxyzz
fxxyzz = create_derivative(f(x,y,z), (x,x,y,z,z))

# xyyzz
fxyyzz = create_derivative(f(x,y,z), (x,y,y,z,z))


###########################################
# Order 6
#xxyyzz

fxxyyzz = create_derivative(f(x,y,z), (x,x,y,y,z,z))



# Test Function Lists

test_order_zero_one_two = [f, fx, fy, fz, fxx, fxy, fxz, fyy, fyz, fzz]  # Length 10

test_order_three_ex = [fxxy, fxxz, fxyy, fxyz, fyyz, fxzz, fyzz]     # Length 7,     Ex triplets of one variable.

test_order_four_ex = [fxxyy, fxxzz, fyyzz, fxxyz, fxyyz, fxyzz]      # Length 6

test_order_five_ex = [fxxyyz, fxxyzz, fxyyzz]                       # Length 3

test_order_six_ex = [fxxyyzz]                                      # Length 1




# Norm 1
def norm1(vector):
    y = 0
    for i in vector:
        y += abs(i)
    return y

# Norm 2
def norm2(vector):
    y = 0
    for i in vector:
        y += i**2
    return y**0.5


In [6]:
##########################


# Sample info

#  Box input


def get_box_function():
    """
    Takes in 2 x-coords, 2 y-coords, and 2 z-coords from the user.
    
    Returns a list of the 8 points that form the box and a list
    of the delta's for each coordinate.
    """ 
    # Empty list for storing user input.
    xcoords = []
    ycoords = []
    zcoords = []
    
    # Is the box the unit cube?  Or are we using an arbitrary box?
    is_cube = input('Is sample box the unit cube?  (y/n)')
    
    if is_cube == 'y':
        xcoords = [0, 1]
        ycoords = [0, 1]
        zcoords = [0, 1]
        
        
    if is_cube == 'n':
        # Gather user input. Stiore them in list.
        xcoords.append(float(input('Enter the first x coordinate')))
        xcoords.append(float(input('Enter the second x coordinate')))

        ycoords.append(float(input('Enter the first y coordinate')))
        ycoords.append(float(input('Enter the second y coordinate')))

        zcoords.append(float(input('Enter the first z coordinate')))
        zcoords.append(float(input('Enter the second z coordinate')))
    
    if (is_cube != 'y') & (is_cube != 'n'):
        raise Exception('Please input y/n to declare that the unit cube is being used.')
    
    # Compute the range in each coordinate as delta_variable.
    x_delta = max(xcoords) - min(xcoords)
    y_delta = max(ycoords) - min(ycoords)
    z_delta = max(zcoords) - min(zcoords)
    
    deltas = [x_delta, y_delta, z_delta]
    
    # Generate all 8 points.
    points_list = [ (i, j, k) for k in zcoords for j in ycoords for i in xcoords]
        
    return points_list, deltas, min(xcoords), min(ycoords), min(zcoords)
    


In [7]:
# Assign box paramters  

test_box, deltas, xmin, ymin, zmin = get_box_function()

# Create the function for transformation.
def to_cube(x, y, z):
    """
    Takes in an (x,y,z) tuple from any place in the coordinate system and applies the to_cube trasnformation which
    maps the sample box to the unit cube.
    """
    xnew = (x - xmin)/deltas[0]
    ynew = (y - ymin)/deltas[1]
    znew = (z - zmin)/deltas[2]
    
    return (xnew, ynew, znew)
    

Is sample box the unit cube?  (y/n) y


In [19]:
# Functions to use and their tuple of variables that they are derivatives for.  To be used in the transformed box version.

# Case 2 dictionary.

case_2_functions_dict = {   f:(), fx:(x,), fy:(y,), fz:(z,), fxx:(x,x), fxy:(x,y), fxz:(x,z), fyy:(y,y), fyz:(y,z), fzz:(z,z),  # Orders 0-2, Length 10
                fxxy:(x,x,y), fxxz:(x,x,z), fxyy:(x,y,y), fxyz:(x,y,z), fyyz:(y,y,z), fxzz:(x,z,z), fyzz:(y,z,z),    # Length 7,   3rd order  Ex triplets of one variable.
                fxxyy:(x,x,y,y), fxxzz:(x,x,z,z), fyyzz:(y,y,z,z), fxxyz:(x,x,y,z), fxyyz:(x,y,y,z), fxyzz:(x,y,z,z),    # Order 4,  Length 6
                fxxyyz:(x,x,y,y,z), fxxyzz:(x,x,y,z,z), fxyyzz:(x,y,y,z,z),                     # Order 5, Length 3
                fxxyyzz:(x,x,y,y,z,z)}                                 # Length 1



def delta_scalar(variables:tuple, deltas:list)->float:
    """
    Takes in a tuple of variables and a list of deltas for each variable.
    
    Returns the product of the delta scalars based on the tuple of variables.
    """
    scalar = 1
    
    for i in variables:
        
        if i==x:
            scalar *= deltas[0]  # use delta x
        
        if i==y:
            scalar *= deltas[1]
        
        if i==z:
            scalar *= deltas[2]
    
    return scalar    



def get_fitted_coeffs(matrix, functions_dict:dict, test_box:list)->list:
    """
    Takes in the matrix for the unfitted polynomial, the list of sample functions (RHS).
    
    Scales the derivatives and solves the system
    
    matrix * alpha_coeffs = delta scalars * RHS
    
    for the alpha coefficients.  Returns the list of fitted alpha coefficients.
    """
    
    # We evaluate the test function and its derivatives at the eight corners.
    #  We then form the RHS vector for our linear system.
    rhs = np.array([ delta_scalar(variables, deltas) * f(*corner) for f, variables in functions_dict.items() for corner in test_box ], dtype='float')

    #  Solving the system for the Values of Alpha coefficients
    fitted_alpha_coefficients = la.solve(matrix, rhs)
    
    return fitted_alpha_coefficients


new_alpha_coeffs = get_fitted_coeffs(case2_matrix, case_2_functions_dict, test_box)




#  Construction of the fitted polynomials.

#  Construction of Polynomial p(x,y)

def fit_p(x,y,z):
    preimage_polynomial = create_polynomial(new_alpha_coeffs, variables)
    
    return preimage_polynomial(*to_cube(x,y,z))


# px(x,y,z)
fit_px = create_derivative(fit_p(x,y,z), (x,))

# p(x,y,z) y derivative
fit_py = create_derivative(fit_p(x,y,z), (y,))

# p(x,y,z) z derivative
fit_pz = create_derivative(fit_p(x,y,z), (z,))

# pxy(x,y,z)  The xy derv
fit_pxy = create_derivative(fit_p(x,y,z), (x,y))

# pxz(x,y,z)  The xz derv
fit_pxz = create_derivative(fit_p(x,y,z), (x,z))

# pyz(x,y,z)  The yz derv
fit_pyz = create_derivative(fit_p(x,y,z), (y,z))

# pzz(x,y,z)  The zz derv
fit_pzz = create_derivative(fit_p(x,y,z), (z,z))

# pzzz(x,y,z)  The zzz derv
fit_pzzz = create_derivative(fit_p(x,y,z), (z,z,z))




# pxyz(x,y,z)  The xyz derv
fit_pxyz = create_derivative(fit_p(x,y,z), (x,y,z))



In [ ]:
f(1.1, 1.2, 1.1)

In [ ]:
fit_p(1.1, 1.2, 1.1)

In [9]:
#  Upper Polynomial  (For numerical proof of continuity)


# Assign box paramters

upper_box, up_deltas, up_xmin, up_ymin, up_zmin = get_box_function()


Is sample box the unit cube?  (y/n) n
Enter the first x coordinate 0
Enter the second x coordinate 1
Enter the first y coordinate 0
Enter the second y coordinate 1
Enter the first z coordinate 1
Enter the second z coordinate 2


In [10]:
# Create the function for transformation.
def upper_to_cube(x, y, z):
    """
    Takes in an (x,y,z) tuple from any place in the coordinate system and applies the to_cube trasnformation which
    maps the sample box to the unit cube.
    """
    xnew = (x - up_xmin)/up_deltas[0]
    ynew = (y - up_ymin)/up_deltas[1]
    znew = (z - up_zmin)/up_deltas[2]
    
    return (xnew, ynew, znew)
    


    

upper_alpha_coeffs = get_fitted_coeffs(case2_matrix, case_2_functions_dict, upper_box)

In [20]:
    

#  Need to change to_cube function.

def upper_function(x,y,z):
    preimage_polynomial = create_polynomial(upper_alpha_coeffs, variables)
    
    return preimage_polynomial(*upper_to_cube(x,y,z))


# px(x,y,z)
upper_fit_px = create_derivative(upper_function(x,y,z), (x,))

# p(x,y,z) y derivative
upper_fit_py = create_derivative(upper_function(x,y,z), (y,))

# p(x,y,z) z derivative
upper_fit_pz = create_derivative(upper_function(x,y,z), (z,))

# pxy(x,y,z)  The xy derv
upper_fit_pxy = create_derivative(upper_function(x,y,z), (x,y))

# pxz(x,y,z)  The xz derv
upper_fit_pxz = create_derivative(upper_function(x,y,z), (x,z))

# pzz(x,y,z)  The zz derv
upper_fit_pzz = create_derivative(upper_function(x,y,z), (z,z))

# pzzz(x,y,z)  The zzz derv
upper_fit_pzzz = create_derivative(upper_function(x,y,z), (z,z,z))



# pyz(x,y,z)  The yz derv
upper_fit_pyz = create_derivative(upper_function(x,y,z), (y,z))

# pxyz(x,y,z)  The xyz derv
upper_fit_pxyz = create_derivative(upper_function(x,y,z), (x,y,z))

In [23]:
# upper box   (Same coordinate system not cube)

upper_fit_pxyz(.5, .5, 1)


-0.44515886111365655

In [24]:
# lower

fit_pxyz(.5, .5, 1)


-0.44515886111365194

In [ ]:
#  Section 7 Evaluation Section


In [ ]:
# Sample points for testing

test_points = [(0.5, 0.25, 0.25), (0.4, 0.8, 0.1),
               (0.5, 0.5, 0.25), (0.8, 0.9, 0.7)]

#  Estimates and Error.

# Actual values for test function number 1.
actual_f1_values = np.array([ f(*point) for point in test_points])   # The * tells Python to process the tuple as coordinates.

# Estimated values from our fitted polynomial.
estimated_values = np.array([ fit_p(*point) for point in test_points])

# Error vector.
error_f1 = actual_f1_values - estimated_values


print('The 1-norm of the error vector is: ', norm1(error_f1), '\n')

print('The 2-norm of the error vector is: ', norm2(error_f1), '\n')



In [ ]:
#  Evaluation section

In [ ]:
transformed_fit_p(x,y,z)

In [ ]:
# Actual

fx(.1, .8, .7)

In [ ]:
fit_px(.1, .8, .7)

In [ ]:
# Estimate



In [ ]:
# Actual

fxyz(1.11, 2.11, 3.11)

In [ ]:
############################################################################3

#  Continuity Examination Section


# 